In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits

#fits_image_filename = fits.util.get_testdata_filepath()

In [2]:
_epsilon = 3e-4 

In [4]:
hdul_gev = fits.open('data/gll_psc_v16.fit')
cat_gev = hdul_gev[1].data

In [5]:
names_gev = [
    'ASSOC_TEV', 
    'CLASS1', 
    'GLON', 
    'GLAT', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000',  
    'Flux30_100',
    ] 

In [7]:
hdul_tev = fits.open('data/gammacat.fits.gz')
cat_tev = hdul_tev[1].data

In [8]:
names_tev = [
    'classes', 
    'glat', 
    'glon', 
    'morph_pa', 
    'sed_dnde', 
    'sed_dnde_err', 
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err'
    ]

In [9]:
def corresponding_bins(cat_gev, cat_tev, epsilon):
    d = {}
    
    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for j in range(len(glat_tev)):
        for i in range(len(glat_gev)):
            if ((class_tev[j].find('bin') != -1) 
                & ((class_gev[i].find('BIN') != -1) | (class_gev[i].find('HMB') != -1))
                & (np.abs(glat_gev[i] - glat_tev[j]) < epsilon) 
                & (np.abs(glon_gev[i] - glon_tev[j]) < epsilon)):
                d.update({j : i})
    return d
                
print(corresponding_bins(cat_gev, cat_tev, 1))

{13: 319, 45: 1172, 125: 2358}


In [10]:
gevToTev = {'BLL': 'blazar', 
            'FRSQ': 'frsq', 
            'HMB': 'bin' , 
            'BIN': 'bin', 
            'GAL': 'galaxy', 
            'PSR': 'psr', 
            'PWN': 'pwn', 
            'SNR': 'snr', 
            '': 'unid'}
tevToGev = {v:k for k, v in gevToTev.items()}

In [11]:
def common(cat_gev, cat_tev, epsilon):
    """This function looks for the same objects in GeV and TeV catalogs
    
    Return: dictionary with corresponding values
    
    feature1(list) - features of GeV 
    feature2(list) - features of TeV 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}

    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for i in range(len(glat_gev)):
        for j in range(len(glat_tev)):
            classGeV = class_gev[i]
            start = -1
            try: 
                start = (class_tev[j].find(gevToTev[classGeV]))
            except KeyError:
                continue
            if (start != -1):
                if ((glat_gev[i] - glat_tev[j] < epsilon) and (glon_gev[i] - glon_tev[j] < epsilon)) :
                    d.update({j : i})
    return d

D = common(cat_gev, cat_tev, 3e-10)
print(D)

{95: 3030, 103: 3030, 105: 3030, 107: 3030, 108: 3030, 109: 3030, 11: 3030, 46: 3030, 68: 3030, 76: 3030, 77: 3030, 84: 3030, 86: 3030, 88: 3030, 90: 3030, 91: 3030, 93: 3030, 24: 2994, 31: 2994, 152: 3030, 153: 3030, 154: 3030, 155: 3030, 156: 3030, 9: 2884, 41: 2884, 37: 620, 44: 955, 47: 955, 50: 955, 61: 2354, 64: 955, 65: 2405, 66: 2405, 74: 955, 79: 1847, 82: 1847, 85: 2405, 87: 2405, 89: 2405, 94: 955, 100: 1847, 29: 2699, 39: 2699, 73: 2699, 75: 2699, 81: 2699, 96: 2699, 97: 2699, 98: 2699, 99: 2699, 104: 2699, 70: 2699, 60: 2358, 120: 2926, 113: 2704, 117: 2926, 126: 2926, 128: 2926, 134: 2926, 135: 2926, 111: 2237, 118: 2926, 119: 2926, 123: 2926, 127: 2926, 129: 2926, 131: 2926, 133: 2926, 137: 2926, 139: 2926, 140: 2926, 144: 2984, 141: 2926, 142: 2984, 0: 2699, 2: 2699, 115: 2274, 122: 2286, 143: 2440, 146: 2499, 151: 2699, 159: 2699, 160: 2699, 8: 2405, 132: 2354, 136: 2405, 138: 2405, 13: 2358, 163: 2699}


In [12]:
names_common = [
    'glat',
    'glon',
    'morph_pa',
    'sed_dnde',
    'sed_dnde_err',
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err', 
    'ASSOC_TEV', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'CLASS1']

In [13]:
d_coord = {'glon' : 'GLON', 'glat' : 'GLAT'}

In [14]:
def create_common_data(cat_gev, cat_tev, D, namefinal):
    """The fonction adds objects found both in GeV and TeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    namefinal(list) - names of required features to fill in data
    """
    k = 0
    data = []
    
    for j in D.keys():
        data.append([])
        for i in range(13):
            data[k].append(cat_gev[namefinal[i]][j])
        for i in range(13, len(namefinal)):
            data[k].append(cat_tev[namefinal[i]][D[j]])
        k = k+1
        
    df_common = pd.DataFrame(data = data, columns = namefinal)
        
    return df_common
    #print(str(cat['CLASS1'][D[i]]) + " - " + str(cat2['classes'][i]) + "\n")

In [15]:
def create_only_tev_data(cat_tev, D, name_tev):
    """The fonction adds objects found only in TeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_tev(list) - names of TeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_tev)):
        if not(j in D.keys()):
            data.append([])
            for i in range(len(name_tev)):
                data[k].append(cat_tev[name_tev[i]][j])
            k = k+1
            
    df_only_tev = pd.DataFrame(data = data, columns = name_tev)
        
    return df_only_tev

In [16]:
def create_only_gev_data(cat_gev, D, names_gev):
    """The fonction adds objects found only in GeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_gev(list) - names of GeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_gev)):
        if not(j in D.values()):
            data.append([])
            for i in range(len(names_gev)):
                data[k].append(cat_gev[names_gev[i]][j])
            k = k+1
            
    df_only_gev = pd.DataFrame(data = data, columns = names_gev)
        
    return df_only_gev

In [17]:
data = []
common_data = create_common_data(cat_tev, cat_gev, D, names_common)
print("Added common values = " + str(len(common_data)))
only_tev_data = create_only_tev_data(cat_tev, D, names_tev)
print("Added values which can be found only in grammacat.fits (TeV) = " + str(len(only_tev_data)))
only_gev_data = create_only_gev_data(cat_gev, D, names_gev)
print("Added values which can be found only in gll_psc_v16.fit (GeV) = " + str(len(only_gev_data)))

print("Size of grammacat.fits (TeV) = "  + str(len(cat_tev)))
print("Size of gll_psc_v16.fits (GeV) = "  + str(len(cat_gev)))

Added common values = 90
Added values which can be found only in grammacat.fits (TeV) = 76
Added values which can be found only in gll_psc_v16.fit (GeV) = 3016
Size of grammacat.fits (TeV) = 166
Size of gll_psc_v16.fits (GeV) = 3034


In [18]:
common_data.head()

glat        glon  morph_pa  \
0 -0.468805  345.682251     -20.0   
1 -0.005492  353.410187       NaN   
2  0.190997  358.396912       NaN   
3 -0.578284  358.873413       NaN   
4 -0.046162  359.944244       NaN   

                                            sed_dnde  \
0  [8.11092e-12, 1.61977e-12, 4.62848e-13, 7.2757...   
1  [2.5207e-12, 5.79489e-13, 7.97074e-14, 5.29676...   
2  [1.537e-12, 4.551e-13, 7.933e-14, 2.303e-14, 1...   
3  [5.27582e-11, 1.08499e-11, 2.385e-12, 4.71598e...   
4  [1.42756e-10, 7.84682e-11, 4.92253e-11, 3.1732...   

                                        sed_dnde_err  \
0  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
1  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
2  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
3  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
4  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   

                                           sed_e_ref  spec_dnde_1TeV  \
0  [0.878584, 1.56, 2.7675, 4.90964, 8.7023, 15.4...    5.596000e-12   
1  [0.344872, 0.792055, 2.07944, 5.43807, 13.8391...    3.146732e-13   
2  [0.4897, 0.7662, 1.327, 2.34, 4.146, 7.375, 13...    2.100000e-13   
3  [0.34603, 0.604812, 1.06183, 1.87934, 3.32625,...    2.840000e-12   
4  [0.165368, 0.204729, 0.251321, 0.307177, 0.374...    2.322480e-12   

   spec_dnde_1TeV_err  spec_eflux_1TeV_10TeV  spec_eflux_1TeV_10TeV_err  \
0        6.404000e-13           1.139231e-11               1.676283e-12   
1        5.055759e-14           8.918628e-13               2.137956e-13   
2        4.000000e-14           5.594303e-13               1.561200e-13   
3        2.300000e-13           5.159108e-12               6.384828e-13   
4        5.452722e-14           5.835075e-12               3.561090e-13   

    ...    Flux300_1000  Flux30_100      Flux1000 Flux10000_100000  \
0   ...    1.920689e-10         NaN  3.016442e-10     3.124023e-11   
1   ...    1.920689e-10         NaN  3.016442e-10     3.124023e-11   
2   ...    1.920689e-10         NaN  3.016442e-10     3.124023e-11   
3   ...    1.920689e-10         NaN  3.016442e-10     3.124023e-11   
4   ...    1.920689e-10         NaN  3.016442e-10     3.124023e-11   

   Flux1000_3000   Flux100_300  Flux3000_10000  Flux300_1000  Flux30_100  \
0   1.822992e-10  1.477740e-09    1.212272e-10  1.920689e-10         NaN   
1   1.822992e-10  1.477740e-09    1.212272e-10  1.920689e-10         NaN   
2   1.822992e-10  1.477740e-09    1.212272e-10  1.920689e-10         NaN   
3   1.822992e-10  1.477740e-09    1.212272e-10  1.920689e-10         NaN   
4   1.822992e-10  1.477740e-09    1.212272e-10  1.920689e-10         NaN   

   CLASS1  
0          
1          
2          
3          
4          

[5 rows x 30 columns]

In [19]:
only_tev_data.head()

classes       glat        glon  morph_pa  \
0     hbl -78.086937   74.632011       NaN   
1     hbl -81.216103   94.174644       NaN   
2     hbl  -2.978120  120.975426       NaN   
3  galaxy -87.964546   97.363853       NaN   
4     hbl -22.939846  132.416458       NaN   

                                            sed_dnde  \
0  [1.907e-12, 1.462e-13, 8.40099e-15, nan, nan, ...   
1  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
2  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
3  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
4  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   

                                        sed_dnde_err  \
0  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
1  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
2  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
3  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
4  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   

                                           sed_e_ref  spec_dnde_1TeV  \
0  [0.4452, 0.949698, 2.121, nan, nan, nan, nan, ...    1.175430e-13   
1  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...             NaN   
2  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...             NaN   
3  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...             NaN   
4  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...             NaN   

   spec_dnde_1TeV_err  spec_eflux_1TeV_10TeV  spec_eflux_1TeV_10TeV_err  \
0        4.592630e-14           1.291623e-13               8.729060e-14   
1                 NaN                    NaN                        NaN   
2                 NaN                    NaN                        NaN   
3                 NaN                    NaN                        NaN   
4                 NaN                    NaN                        NaN   

   spec_flux_1TeV  spec_flux_1TeV_crab  spec_flux_1TeV_crab_err  
0    4.897623e-14             0.236093                 0.136923  
1             NaN                  NaN                      NaN  
2             NaN                  NaN                      NaN  
3             NaN                  NaN                      NaN  
4             NaN                  NaN                      NaN

In [20]:
only_gev_data.head()

ASSOC_TEV CLASS1        GLON       GLAT  Variability_Index      Flux1000  \
0                   117.693878   3.402958          40.753517  1.015962e-09   
1                   345.410522 -74.946754          44.845318  2.038660e-10   
2              spp  117.293091   0.925701          53.047859  6.403683e-10   
3              bll   89.022202 -67.324249          49.738213  6.952190e-10   
4             fsrq  107.665428 -40.047157         130.336334  2.944868e-10   

   Flux10000_100000  Flux1000_3000   Flux100_300  Flux3000_10000  \
0      2.839629e-11   1.236649e-09  1.808316e-08    5.780520e-11   
1      1.590734e-11   1.450903e-10  9.876651e-11    7.452389e-11   
2      2.021964e-12   3.020877e-10  2.913030e-08    1.158661e-10   
3      1.005633e-11   5.223298e-10  4.086941e-09    2.106892e-10   
4      1.035538e-14   3.565206e-10  1.523905e-08    8.413887e-15   

   Flux300_1000  Flux30_100  
0  6.940733e-09         NaN  
1  1.342487e-10         NaN  
2  4.650789e-09         NaN  
3  2.083574e-09         NaN  
4  3.639644e-09         NaN